Save and Restore a model using TensorFlow. This example is using the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/).

In [1]:
from __future__ import print_function

from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/tmp/data", one_hot=True)

import tensorflow as tf

W0822 15:43:01.066257 139766011230080 deprecation.py:323] From <ipython-input-1-c644ebdb1629>:4: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0822 15:43:01.068099 139766011230080 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0822 15:43:01.073290 139766011230080 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for 

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz


W0822 15:43:01.559355 139766011230080 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0822 15:43:01.563185 139766011230080 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0822 15:43:01.692362 139766011230080 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: __init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a f

Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
#training hyperparameters
learning_rate= 0.001
batch_size = 100
display_step = 1
model_path="/tmp/model.ckpt"

# network hyperparameters
n_hidden_1=256
n_hidden_2=256
n_input=784
n_classes=10

#graph input
x=tf.placeholder("float", [None, n_input])
y=tf.placeholder("float", [None, n_classes])

In [0]:
#create model
def multilayer_perceptron(x, weights, biases):
  #hidden layer with RELU activation
  layer_1=tf.add(tf.matmul(x, weights['h1']), biases['b1'])
  layer_1=tf.nn.relu(layer_1)
  #hidden layer with RELU activation
  layer_2=tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
  layer_2=tf.nn.relu(layer_2)
  #output layer with linear activation
  out_layer=tf.matmul(layer_2, weights['out'])+biases['out']
  return out_layer

In [0]:
#store layer weights and bias
weights = {
    'h1':tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2':tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out':tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases={
    'b1':tf.Variable(tf.random_normal([n_hidden_1])),
    'b2':tf.Variable(tf.random_normal([n_hidden_2])),
    'out':tf.Variable(tf.random_normal([n_classes]))
}

In [5]:
#construct model
pred=multilayer_perceptron(x, weights, biases)

#define loss and optimizer
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

#initialize variables
init=tf.global_variables_initializer()


# saver op to save and restore all the variables
saver = tf.train.Saver()

W0822 16:35:09.828020 139766011230080 deprecation.py:323] From <ipython-input-5-5ecde184d0e1>:4: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [7]:
#running first session
print("starting 1st session....")
with tf.Session() as sess:
  sess.run(init)
  
  #training cycle
  for epoch in range(3):
    avg_cost=0.0
    total_batch=int(mnist.train.num_examples/batch_size)
    #loop over all batches
    for i in range(total_batch):
      batch_x, batch_y = mnist.train.next_batch(batch_size)
      #run optimization (op) and cost op
      _, c = sess.run([optimizer, cost], feed_dict={x:batch_x, y:batch_y})
      
      #compute average loss
      avg_cost += c/total_batch
      
    # display logs per epoch step
    if epoch%display_step==0:
      print("epoch", epoch+1, "cost", avg_cost)
      
  print("first optimization finished")
  
  #test model
  correct_prediction=tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
  
  #calculate accuracy
  accuracy=tf.reduce_mean(tf.cast(correct_prediction, "float"))
  print("accuracy",accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))
  
  #save model weights to disk
  save_path = saver.save(sess, model_path)
  print("model saved in file:", save_path)

starting 1st session....
epoch 1 cost 185.52472235419546
epoch 2 cost 40.62224635644396
epoch 3 cost 25.350549232743006
first optimization finished
accuracy 0.9065
model saved in file: /tmp/model.ckpt


In [8]:
#running new session
print("starting 2nd session...")
with tf.Session() as sess:
  sess.run(init)
  
  #restore model weights from previously saved model
  load_path = saver.restore(sess, model_path)
  print("model restroed from file", save_path)
  
  #resuem training
  for epoch in range(7):
    avg_cost=0.0
    total_batch=int(mnist.train.num_examples/batch_size)
    #loop over all batches
    for i in range(total_batch):
      batch_x, batch_y = mnist.train.next_batch(batch_size)
      #run optimization op and cost op
      _, c = sess.run([optimizer, cost], feed_dict={x:batch_x, y:batch_y})
      
      #compute average loss
      avg_cost += c/total_batch
    
    #display logs per epoch step
    if epoch%display_step==0:
      print("epoch",epoch+1, "cost",avg_cost)
      
  print("second optimization finished")
  
  #test model
  correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
  #calculate accuracy
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
  
  print("Accuracy:",accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

W0822 16:50:46.534085 139766011230080 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


starting 2nd session...
model restroed from file /tmp/model.ckpt
epoch 1 cost 17.685845511609863
epoch 2 cost 12.991956444694233
epoch 3 cost 9.754479980147009
epoch 4 cost 7.305098009221114
epoch 5 cost 5.644987883518022
epoch 6 cost 4.250406343949144
epoch 7 cost 3.2390454876223704
second optimization finished
Accuracy: 0.9324
